# Finetune Embedding Model

此笔记本包含使用 Uniem 库对模型进行微调的示例，在运行此 notebook 之前，我们需要先安装了 Uniem 库，安装方式如下。

```bash
pip install uniem
```

In [ ]:
!pip install uniem

**微调模型的方式非常简单，只需要按照指定的格式准备数据就可以了，其余的事情都将由 uniem 替你完成，几行代码就够了，就像下面👇🏻这样**

```python
from datasets import load_dataset
from uniem.finetuner import FineTuner

dataset = load_dataset('shibing624/nli_zh', 'STS-B')
finetuner = FineTuner('moka-ai/m3e-small', dataset=dataset)
finetuner.run(epochs=1)
```

## 数据准备

你需要准备 `uniem` 支持的数据格式的数据集

目前，`uniem` 支持一下三种格式

1. `PairRecord`
2. `TripletRecord`
3. `ScoredPairRecord`

In [1]:
import os
import warnings
from uniem.data_structures import RecordType, PairRecord, TripletRecord, ScoredPairRecord

os.environ['TOKENIZERS_PARALLELISM'] = 'false'
warnings.filterwarnings('ignore')
print(f'record_types: {[record_type.value for record_type in RecordType]}')

record_types: ['pair', 'triplet', 'scored_pair']


### PairRecord

当你的数据集中只有问答对或者相似句子的正例

In [3]:
pair_record = PairRecord(text='肾结石如何治疗？', text_pos='如何治愈肾结石')
print(f'pair_record: {pair_record}')

pair_record: PairRecord(text='肾结石如何治疗？', text_pos='如何治愈肾结石')


### TripletRecord

当你的数据集中同时有问答或者句子的正例和负例

In [4]:
triplet_record = TripletRecord(text='肾结石如何治疗？', text_pos='如何治愈肾结石', text_neg='胆结石有哪些治疗方法？')
print(f'triplet_record: {triplet_record}')

triplet_record: TripletRecord(text='肾结石如何治疗？', text_pos='如何治愈肾结石', text_neg='胆结石有哪些治疗方法？')


### ScoredPairRecord

当你的数据集是一个二分类任务，或者能给出实际相似分数

#### 二分类任务

In [5]:
scored_pair_record1 = ScoredPairRecord(sentence1='肾结石如何治疗？', sentence2='如何治愈肾结石', label=1.0)
scored_pair_record2 = ScoredPairRecord(sentence1='肾结石如何治疗？', sentence2='胆结石有哪些治疗方法？', label=0.0)
print(f'scored_pair_record: {scored_pair_record1}')
print(f'scored_pair_record: {scored_pair_record2}')

scored_pair_record: ScoredPairRecord(sentence1='肾结石如何治疗？', sentence2='如何治愈肾结石', label=1.0)
scored_pair_record: ScoredPairRecord(sentence1='肾结石如何治疗？', sentence2='胆结石有哪些治疗方法？', label=0.0)


#### 相似分数

In [6]:
scored_pair_record1 = ScoredPairRecord(sentence1='肾结石如何治疗？', sentence2='如何治愈肾结石', label=2.0)
scored_pair_record2 = ScoredPairRecord(sentence1='肾结石如何治疗？', sentence2='胆结石有哪些治疗方法？', label=1.0)
scored_pair_record3 = ScoredPairRecord(sentence1='肾结石如何治疗？', sentence2='失眠如何治疗', label=0)
print(f'scored_pair_record: {scored_pair_record1}')
print(f'scored_pair_record: {scored_pair_record2}')
print(f'scored_pair_record: {scored_pair_record3}')

scored_pair_record: ScoredPairRecord(sentence1='肾结石如何治疗？', sentence2='如何治愈肾结石', label=2.0)
scored_pair_record: ScoredPairRecord(sentence1='肾结石如何治疗？', sentence2='胆结石有哪些治疗方法？', label=1.0)
scored_pair_record: ScoredPairRecord(sentence1='肾结石如何治疗？', sentence2='失眠如何治疗', label=0)


## 示例：STS-B

现在我们假设我们想要 HuggingFace 上托管的 STS-B 数据集上做微调

让我我们先把数据集下载好

In [7]:
from datasets import load_dataset

stsb_dataset_dict = load_dataset('shibing624/nli_zh', 'STS-B')

Found cached dataset nli_zh (/Users/wangyuxin/.cache/huggingface/datasets/shibing624___nli_zh/STS-B/1.0.0/65b555276ee420c801e1c9eb830db959e37f42fa60c68c8b07a4448b8c436706)


  0%|          | 0/3 [00:00<?, ?it/s]

让我们查看一下 STS-B 的数据格式

In [8]:
stsb_dataset_dict['train'][0]

{'sentence1': '一架飞机要起飞了。', 'sentence2': '一架飞机正在起飞。', 'label': 5}

我们发现 STS-B 数据集正好符合我们 `ScoredPairRecord` 的数据格式，因此我们可以直接进行微调不需要任何的数据转换。

我们只需要使用 `uniem` 提供的 `FineTuner` 就可以使用 3 行代码完成微调

注意： `FineTuner` 集成了 HuggingFace 的 datasets 生态，原生支持类型为 `datasets.Dataset` 或者 `datasets.DatasetDict` 的数据集

In [9]:
from datasets import load_dataset

from uniem.finetuner import FineTuner

dataset = load_dataset('shibing624/nli_zh', 'STS-B')
# 指定训练的模型为 m3e-small
finetuner = FineTuner('moka-ai/m3e-small', dataset=dataset)
finetuner.run(epochs=1)

Found cached dataset nli_zh (/Users/wangyuxin/.cache/huggingface/datasets/shibing624___nli_zh/STS-B/1.0.0/65b555276ee420c801e1c9eb830db959e37f42fa60c68c8b07a4448b8c436706)


  0%|          | 0/3 [00:00<?, ?it/s]

Start with seed: 42
Output dir: finetuned-model
Start training for 1 epochs


  0%|          | 0/163 [00:00<?, ?it/s]

Training finished
Saving model


In [13]:
# 训练过程完成后，会自动保存模型到 finetuned-model 目录下
!ls finetuned-model/model

config.json             special_tokens_map.json tokenizer_config.json
pytorch_model.bin       tokenizer.json          vocab.txt


## 示例： Jsonl 

现在我们要对一个猜谜的数据集进行微调，这个数据集是通过 json line 的形式存储的。

让我先看看数据格式吧。

In [3]:
import pandas as pd

df = pd.read_json('example_data/riddle.jsonl', lines=True)
df.iloc[0].to_dict()

{'instruction': '猜谜语：一身卷卷细毛，吃的青青野草，过了数九寒冬，无私献出白毛。 （打一动物）', 'output': '谜底：白羊'}

这个数据集中，我们有 `instruction` 和 `output` ，我们可以把这两者结合在一起组成一个相似句对。这是一个典型的 `PairRecord` 数据集。

`PairRecord` 需要 `text` 和 `text_pos` 两个字段，因此我们需要对数据集的列进行重新命名，以符合 `PairRecord` 的格式。

In [1]:
import pandas as pd

from uniem.finetuner import FineTuner

# 读取 jsonl 文件
df = pd.read_json('example_data/riddle.jsonl', lines=True)
# 重新命名
df = df.rename(columns={'instruction': 'text', 'output': 'text_pos'})
# 指定训练的模型为 m3e-small
finetuner = FineTuner('moka-ai/m3e-small', dataset=df.to_dict('records'))
finetuner.run(epochs=1, output_dir='finetuned-model-riddle')

Start with seed: 42
Output dir: finetuned-model-riddle
Start training for 1 epochs


  0%|          | 0/31 [00:00<?, ?it/s]

Training finished
Saving model


上面的两个示例分别展示了对 jsonl 本地 `PairRecord`类型数据集，以及 huggingface 远程 `ScoredPair` 类型数据集的读取和训练过程。`TripletRecord` 类型的数据集的读取和训练过程与 `PairRecord` 类型的数据集的读取和训练过程类似，这里就不再赘述了。

也就是说，你只要构造了符合 `uniem` 支持的数据格式的数据集，就可以使用 `FineTuner` 对你的模型进行微调了。

`FineTuner` 接受的 dataset 参数，只要是可以迭代的产生有指定字段（格式）的字典 `dict` 就行了，所以上述示例分别使用 `datasets.DatasetDict` 和 `list[dict]` 两种数据格式。